# 자체 평가지표
* 가지고 있는 제출 파일과 비교

## 데이터 가져오기

In [8]:
import pandas as pd
from tqdm import tqdm

In [9]:
# 판다스 라이브러리에서 과학적 표기 사용 안하기
pd.options.display.float_format = '{:.5f}'.format

In [15]:
# sub24_pycaret12.csv 제출 점수 : 0.1032155541
before_sub = pd.read_csv('sub24_pycaret12.csv')
present_sub = pd.read_csv('sub.csv')

## 열을 일자, 시간, 구분 열 분해

In [16]:
# 전 체출 파일 열 정리
before_sub[['일자', '시간', '구분']] = before_sub['일자|시간|구분'].str.split(' ').to_list()
before_sub = before_sub.drop('일자|시간|구분', axis=1)
before_sub = before_sub[['일자', '시간', '구분', '공급량']]
# 현재 체출 파일 열 정리
present_sub[['일자', '시간', '구분']] = present_sub['일자|시간|구분'].str.split(' ').to_list()
present_sub = present_sub.drop('일자|시간|구분', axis=1)
present_sub = present_sub[['일자', '시간', '구분', '공급량']]

### 평가 지표 함수화

In [17]:
def evaluation_index(before_sub, present_sub):
    eval_set = pd.DataFrame(['MSE','RMSE', 'MAE', 'R2','NMAE'], columns=['평가지표'])
    for division in tqdm(present_sub['구분'].unique()):
        temp_before_sub = before_sub[before_sub['구분'] == division]
        temp_present_sub = present_sub[present_sub['구분'] == division]

        # 전 제출 값 = 실제값 취급
        # 현제 제출 값 = 예측값
        y = temp_before_sub['공급량']
        pred = temp_present_sub['공급량']
        y_mean = y.mean()

        # MSE
        mse = ( ( y - pred) ** 2 ).sum() / len(pred)
            
        # RMSE
        rmse = mse ** (1/2)
        
        # MAE
        mae = ( abs( y - pred ) ).sum() / len(pred)
        
        # R2
        upper = ( (y - pred) ** 2 ).sum()
        lower = ( (y - y_mean) ** 2 ).sum()
        if lower == 0:
            fraction = 0
        else:
            fraction = upper / lower
        r2 = 1 - fraction
        
        #NMAE(Normalized Mean Absolute Error)-정규화 평균 절대 오차 척도
        nmae = ( abs( y - pred ) / y ).sum() / len(pred)
        
        # 평가지표 리스트
        one_eval_set = [mse, rmse, mae, r2, nmae]
        
        # 평가지표 셋에 담기
        eval_set[division] = one_eval_set

    # 평가지표 평균 구하기
    eval_set['평가지표_평균'] = eval_set.mean(axis=1)

    # 평가지표 평균 출력
    print(eval_set[['평가지표', '평가지표_평균']])
    
    return eval_set

In [18]:
set = evaluation_index(before_sub, present_sub)

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 205.54it/s]

   평가지표     평가지표_평균
0   MSE 18974.65673
1  RMSE   118.16060
2   MAE    94.31242
3    R2     0.89797
4  NMAE     0.06540



C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_13368/2173100965.py:41: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  eval_set['평가지표_평균'] = eval_set.mean(axis=1)


In [19]:
set

,평가지표,A,B,C,D,E,G,H,평가지표_평균
0,MSE,25126.92293,18781.67599,130.10962,7561.94613,38338.73351,41797.50777,1085.70113,18974.65673
1,RMSE,158.51474,137.04625,11.40656,86.95945,195.80279,204.44439,32.94998,118.16060
2,MAE,127.18839,108.66072,9.02496,68.83672,156.96860,163.34756,26.15999,94.31242
3,R2,0.89347,0.90403,0.86862,0.90379,0.89693,0.90543,0.91353,0.89797
4,NMAE,0.07337,0.07376,0.05039,0.06657,0.07158,0.05555,0.06661,0.06540
